<a href="https://colab.research.google.com/github/MayerT1/Sewanee_Colab/blob/main/ARISE/4maps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!pip install geemap rasterio geopandas contextily matplotlib pillow

In [5]:
import os
from rasterio.plot import show
import geopandas as gpd
import rasterio
from rasterio.plot import show
from matplotlib.colors import ListedColormap, BoundaryNorm
import numpy as np

# Authenticate and initialize Earth Engine
import ee
import geemap
import geemap.chart as chart
ee.Authenticate()
ee.Initialize(project='servir-sco-assets')
Map = geemap.Map()

In [6]:
from shapely.geometry import box
import geopandas as gpd

minx = -86.8826120358268
miny = 34.12771877579682
maxx = -84.2953317623893
maxy = 36.18887300565079

# Create a Shapely box
bbox_shp = box(minx, miny, maxx, maxy)

# Create a GeoDataFrame
bbox_gdf = gpd.GeoDataFrame({"geometry": [bbox_shp]}, crs="EPSG:4326")


In [7]:


# Paths
ghsl_path = os.path.join('/content/drive/MyDrive/ARSIE_MAP_Creation/urban.tif')
forest_path = os.path.join("/content/drive/MyDrive/ARSIE_MAP_Creation/tree.tif")
eco_path = os.path.join("/content/drive/MyDrive/ARSIE_MAP_Creation/eco.geojson")
counties_path = os.path.join("/content/drive/MyDrive/ARSIE_MAP_Creation/counties_selected.geojson")
tracts_path = os.path.join("/content/drive/MyDrive/ARSIE_MAP_Creation/selected_tracts.geojson")
al_path = os.path.join("/content/drive/MyDrive/ARSIE_MAP_Creation/alabama_outline.geojson")
tn_path = os.path.join("/content/drive/MyDrive/ARSIE_MAP_Creation/tennessee_outline.geojson")
points_path = os.path.join("/content/drive/MyDrive/ARSIE_MAP_Creation/city_points.geojson")

# Read rasters
ghsl_src = rasterio.open(ghsl_path)
forest_src = rasterio.open(forest_path)
# eco_src = rasterio.open(eco_path)

# Read vectors
eco_gdf = gpd.read_file(eco_path).to_crs(epsg=3857)
counties_gdf = gpd.read_file(counties_path).to_crs(epsg=3857)
tracts_gdf = gpd.read_file(tracts_path).to_crs(epsg=3857)
al_gdf = gpd.read_file(al_path).to_crs(epsg=3857)
tn_gdf = gpd.read_file(tn_path).to_crs(epsg=3857)
points_gdf = gpd.read_file(points_path).to_crs(epsg=3857)
bbox3857 = bbox_gdf.to_crs(epsg=3857).geometry.total_bounds  # used for axes limits

print("Loaded files. Extent (EPSG:3857):", bbox3857)


Loaded files. Extent (EPSG:3857): [-9671728.13061778  4045964.45706549 -9383713.40803924  4326641.22492867]


In [8]:
counties_gdf['cat'] = counties_gdf['NAME'].map({
    "Franklin": "Transitional",
    "Grundy": "Distressed",
    "Marion": "Transitional",
    "Sequatchie": "At-Risk",
    "Bledsoe": "Distressed",
    "Warren": "At-Risk",
    "Van Buren": "At-Risk",
    "Coffee": "Transitional",
    "DeKalb": "At-Risk",
    "Marshall": "Transitional",
    "Madison": "Competitive",
    "Jackson": "Transitional"
})

print(counties_gdf)

                                              id           cat  GEOID  \
0   1_1_1_1_1_1_1_1_1_1_1_2_00000000000000000964  Transitional  47051   
1     1_1_1_1_1_1_1_1_1_1_2_0000000000000000093b    Distressed  47061   
2       1_1_1_1_1_1_1_1_1_2_000000000000000004ea  Transitional  47115   
3         1_1_1_1_1_1_1_1_2_000000000000000004eb       At-Risk  47153   
4           1_1_1_1_1_1_1_2_00000000000000000952    Distressed  47007   
5             1_1_1_1_1_1_2_00000000000000000960       At-Risk  47177   
6               1_1_1_1_1_2_00000000000000000944       At-Risk  47175   
7                 1_1_1_1_2_00000000000000000963  Transitional  47031   
8                   1_1_1_2_0000000000000000093e       At-Risk  47041   
9                     1_1_2_000000000000000008da  Transitional  01095   
10                      1_2_000000000000000008df   Competitive  01089   
11                        2_000000000000000004e2  Transitional  01071   

       AWATER LSAD CBSAFP STATEFP COUNTYFP CSAFP F

In [9]:
import matplotlib.pyplot as plt
import contextily as ctx
import geopandas as gpd
import os
import matplotlib.patches as mpatches
import json

# -----------------------------
# Paths
# -----------------------------
out_folder = "/content/drive/MyDrive/ARSIE_MAP_Creation"
os.makedirs(out_folder, exist_ok=True)

# Load the new census tract GeoJSON
tracts_file = "/content/drive/MyDrive/ARSIE_MAP_Creation/selected_tracts.geojson"
tracts_gdf = gpd.read_file(tracts_file).to_crs(epsg=3857)  # project to web mercator for contextily

# -----------------------------
# County categories (all 12 counties)
# -----------------------------
counties_gdf['cat'] = counties_gdf['NAME'].map({
    "Franklin": "Transitional",
    "Grundy": "Distressed",
    "Marion": "Transitional",
    "Sequatchie": "At-Risk",
    "Bledsoe": "Distressed",
    "Warren": "At-Risk",
    "Van Buren": "At-Risk",
    "Coffee": "Transitional",
    "DeKalb": "At-Risk",
    "Marshall": "Transitional",
    "Madison": "Competitive",
    "Jackson": "Transitional"
})

# -----------------------------
# Create figure
# -----------------------------
fig, ax = plt.subplots(figsize=(14,9))
ax.set_title("Primary Study Area — Counties & Selected Tracts", fontsize=12, weight='bold')

# -----------------------------
# Plot counties by category
# -----------------------------
cat_colors = {
    "Distressed":"#ca727f",
    "At-Risk":"#eac69e",
    "Transitional":"#e9ebd4",
    "Competitive":"#c0dad7"
}
for cat, color in cat_colors.items():
    subset = counties_gdf[counties_gdf['cat']==cat]
    subset.plot(ax=ax, facecolor=color, edgecolor='k', linewidth=0.6, alpha=0.6)

# -----------------------------
# Plot updated selected census tracts on top
# -----------------------------
tracts_gdf.plot(ax=ax, facecolor="#f8dd6f", edgecolor='0.15', linewidth=0.6, alpha=0.5)

# -----------------------------
# Legend only for counties and tracts
# -----------------------------
handles = [mpatches.Patch(color=v, label=k, alpha=0.7) for k,v in cat_colors.items()]
handles.append(mpatches.Patch(color="#f8dd6f", label="Selected Census Tracts FY26", alpha=0.6))
ax.legend(handles=handles, loc='lower right', frameon=True, fontsize=8)

# -----------------------------
# Add OSM basemap with dimmed colors
# -----------------------------
minx, miny, maxx, maxy = counties_gdf.total_bounds  # use counties for overall extent
# Slight buffer to zoom out and avoid cutting off counties
x_buffer = (maxx - minx) * 0.04
y_buffer = (maxy - miny) * 0.04
ax.set_xlim(minx - x_buffer, maxx + x_buffer)
ax.set_ylim(miny - y_buffer, maxy + y_buffer)

ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik, zoom=9, alpha=0.5)

# -----------------------------
# Remove coordinates / ticks
# -----------------------------
ax.set_xticks([])
ax.set_yticks([])
ax.set_xlabel('')
ax.set_ylabel('')

# -----------------------------
# Save PNG
# -----------------------------
main_map_file = os.path.join(out_folder, "main_map_counties_tracts.png")
plt.savefig(main_map_file, dpi=300, bbox_inches='tight')
plt.close(fig)
print("✅ Saved main map with updated census tracts:", main_map_file)


✅ Saved main map with updated census tracts: /content/drive/MyDrive/ARSIE_MAP_Creation/main_map_counties_tracts.png


forst inlay

In [10]:
import matplotlib.pyplot as plt
import geopandas as gpd
import os
import rasterio
import numpy as np

# -----------------------------
# Paths
# -----------------------------
out_folder = "/content/drive/MyDrive/ARSIE_MAP_Creation"
os.makedirs(out_folder, exist_ok=True)

forest_path = "/content/drive/MyDrive/ARSIE_MAP_Creation/tree.tif"
forest_src = rasterio.open(forest_path)

# -----------------------------
# County categories (all 12 counties)
# -----------------------------
counties_gdf['cat'] = counties_gdf['NAME'].map({
    "Franklin": "Transitional",
    "Grundy": "Distressed",
    "Marion": "Transitional",
    "Sequatchie": "At-Risk",
    "Bledsoe": "Distressed",
    "Warren": "At-Risk",
    "Van Buren": "At-Risk",
    "Coffee": "Transitional",
    "DeKalb": "At-Risk",
    "Marshall": "Transitional",
    "Madison": "Competitive",
    "Jackson": "Transitional"
})

# Keep only counties of interest
selected_counties_gdf = counties_gdf[counties_gdf['cat'].notnull()]

# -----------------------------
# Reproject counties to match raster CRS
# -----------------------------
counties_for_plot = selected_counties_gdf.to_crs(forest_src.crs)

# -----------------------------
# Read forest raster at native resolution
# -----------------------------
forest_img = forest_src.read(1)
forest_img = np.where(forest_img < 0, np.nan, forest_img)  # mask no-data
forest_extent = rasterio.plot.plotting_extent(forest_src)

# -----------------------------
# Create figure
# -----------------------------
fig, ax = plt.subplots(figsize=(14,9))
ax.set_title("Forest Cover with Counties Overlay", fontsize=12, weight='bold')

# Plot forest raster
im = ax.imshow(forest_img, extent=forest_extent, origin='upper', cmap='Greens', vmin=0, vmax=100)

# Colorbar
cbar = fig.colorbar(im, ax=ax, fraction=0.045, pad=0.01)
cbar.set_label("Tree cover %", fontsize=8)

# Overlay counties (hollow, black border)
counties_for_plot.plot(ax=ax, facecolor="none", edgecolor='black', linewidth=3.0)

# Slight buffer around counties for nicer view
minx, miny, maxx, maxy = counties_for_plot.total_bounds
x_buffer = (maxx - minx) * 0.04
y_buffer = (maxy - miny) * 0.04
ax.set_xlim(minx - x_buffer, maxx + x_buffer)
ax.set_ylim(miny - y_buffer, maxy + y_buffer)

# Remove axes labels / ticks
ax.set_xticks([])
ax.set_yticks([])
ax.set_xlabel('')
ax.set_ylabel('')

# Save PNG
forest_map_file = os.path.join(out_folder, "forest_map_counties_overlay.png")
plt.savefig(forest_map_file, dpi=300, bbox_inches='tight')
plt.close(fig)
print("✅ Saved forest map with counties overlay:", forest_map_file)


✅ Saved forest map with counties overlay: /content/drive/MyDrive/ARSIE_MAP_Creation/forest_map_counties_overlay.png


In [ ]:
import matplotlib.pyplot as plt
import geopandas as gpd
import rasterio
import numpy as np
import os
from matplotlib.colors import ListedColormap, BoundaryNorm
import matplotlib.patches as mpatches

# -----------------------------
# Paths
# -----------------------------
out_folder = "/content/drive/MyDrive/ARSIE_MAP_Creation"
os.makedirs(out_folder, exist_ok=True)

urban_path = "/content/drive/MyDrive/ARSIE_MAP_Creation/urban.tif"
urban_src = rasterio.open(urban_path)

# -----------------------------
# County categories (all 12 counties)
# -----------------------------
# counties_gdf
counties_gdf['cat'] = counties_gdf['NAME'].map({
    "Franklin": "Transitional",
    "Grundy": "Distressed",
    "Marion": "Transitional",
    "Sequatchie": "At-Risk",
    "Bledsoe": "Distressed",
    "Warren": "At-Risk",
    "Van Buren": "At-Risk",
    "Coffee": "Transitional",
    "DeKalb": "At-Risk",
    "Marshall": "Transitional",
    "Madison": "Competitive",
    "Jackson": "Transitional"
})

selected_counties = counties_gdf[counties_gdf['cat'].notnull()].to_crs(urban_src.crs)

# -----------------------------
# Load raster and mask values < 4
# -----------------------------
urban = urban_src.read(1)
urban = np.where((urban >= 4) & (urban <= 7), urban, np.nan)

extent = rasterio.plot.plotting_extent(urban_src)

# -----------------------------
# Define explicit colors for classes ≥ 4
# -----------------------------
colors = ["#fee8c8", "#fdbb84", "#e34a33", "#b30000"]  # 4 → 7
cmap = ListedColormap(colors)
norm = BoundaryNorm([3.5, 4.5, 5.5, 6.5, 7.5], cmap.N)

# -----------------------------
# Plot Figure
# -----------------------------
fig, ax = plt.subplots(figsize=(14,9))
ax.set_title("Urban Development Intensity — Study Area", fontsize=14, weight='bold')

im = ax.imshow(urban, extent=extent, origin="upper", cmap=cmap, norm=norm)

# Add legend patches
legend_labels = {
    4: "Low Urban",
    5: "Moderate Urban",
    6: "High Urban",
    7: "Very High Urban"
}
patches = [mpatches.Patch(color=colors[i-4], label=legend_labels[i]) for i in range(4,8)]
ax.legend(handles=patches, loc='lower right', frameon=True, fontsize=9)

# Overlay county boundaries (hollow)
selected_counties.plot(ax=ax, facecolor="none", edgecolor="black", linewidth=1.4)

# Adjust view to counties
minx, miny, maxx, maxy = selected_counties.total_bounds
x_buffer = (maxx - minx) * 0.04
y_buffer = (maxy - miny) * 0.04
ax.set_xlim(minx - x_buffer, maxx + x_buffer)
ax.set_ylim(miny - y_buffer, maxy + y_buffer)

# Remove axis ticks
ax.set_xticks([])
ax.set_yticks([])
ax.set_xlabel("")
ax.set_ylabel("")

# Save
urban_map_file = os.path.join(out_folder, "urban_map_counties_overlay.png")
plt.savefig(urban_map_file, dpi=300, bbox_inches="tight")
plt.close(fig)

print("✅ Saved Urban Intensity Map:", urban_map_file)

